# EU Data compiler

In [ ]:
keywords = ["Sanction*", "international sanction*", "economic sanction*", "economic coercion", "United Nations sanction*", "export control*",
            "export restriction*", "aid withdrawal", "aid AND suspension OR cut OR ban OR freeze OR resumption", "individual sanction","targeted sanction*", "restrictive measure*", 
            "listing*", "Magnitsky sanction*","travel ban", "visa ban", "travel restriction*", "asset freez*", "freezing of bank account*", "prohibition of doing business", 
            "restriction of diplomatic contact*";"human rights violation*", "repression", "atrocity", "instigation of violence", "violence", "murder", "killing*", "torture", 
            "manipulation of elections", "electoral fraud", "democracy support", "nuclear proliferation", "military aggression", "war", "armed conflict", "territorial integrity", "cyber-attack",
            "nuclear OR chemical OR biological AND weapon", "weapons of mass destruction", "proliferation of arms", "disarmament", "terrorist* OR terrorist group OR terrorism", "drug trafficking", 
            "corruption", "human trafficking", "sanctions busting", "violation of sanction*"] 


EUVOC_keywords=[DC_coded == '5788','c_1c478aa5','6288','3870','c_cd74cde9','5124','2219','123','5341', '5688','3483','3219','3454','99','6375','2500','250025']


**Option 1 Python EU lex project (https://pypi.org/project/eurlex/)**

In [11]:
#not availabale via conda or conda forge, also no aadmin right for pip
#! conda install -c conda-forge eurlex
#! conda install -c conda-forge pyeurlex

#load packages from github directly
! pip install git+https://github.com/step21/eurlex.git


  Cloning https://github.com/step21/eurlex.git to c:\users\cecilianatalie.strom\appdata\local\temp\pip-req-build-ontrzlud
  Resolved https://github.com/step21/eurlex.git to commit 9d7c5a9e9669b24a422ed742a021db5ba6963aac
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fire-0.4.0-py2.py3-none-any.whl
  Using cached halo-0.0.31-py3-none-any.whl
  Obtaining dependency information for pdfminer.six<20220525,>=20220524 from https://files.pythonhosted.org/packages/92/1c/f0e976509c8d453cb0f1b360619bbc2b588ef733c32780c9e0221918b7fa/pdfminer.six-20220524-py3-none-any.whl.metadata
  Using cached pdfminer.six-20220524-py3-none-any.whl.metadata (3.7 kB)
  Obtaining dependency information for scriv[toml]<0.1

  Running command git clone --filter=blob:none --quiet https://github.com/step21/eurlex.git 'C:\Users\cecilianatalie.strom\AppData\Local\Temp\pip-req-build-ontrzlud'


*in the next step, take the keyword list and search for each keyword in the EU Lex database, extracct the CELEX numbers of each document for ech search as zipped csv file and compile a list of unique CELEX numbers*

In [4]:
# Define the list of keywords
keywords = [
    "Sanction*", "international sanction*", "economic sanction*", "economic coercion", "United Nations sanction*",
    "export control*", "export restriction*", "aid withdrawal",
    "aid AND suspension OR cut OR ban OR freeze OR resumption", "individual sanction", "targeted sanction*",
    "restrictive measure*", "listing*", "Magnitsky sanction*", "travel ban", "visa ban", "travel restriction*",
    "asset freez*", "freezing of bank account*", "prohibition of doing business", "restriction of diplomatic contact*",
    "human rights violation*", "repression", "atrocity", "instigation of violence", "violence", "murder", "killing*",
    "torture", "manipulation of elections", "electoral fraud", "democracy support", "nuclear proliferation",
    "military aggression", "war", "armed conflict", "territorial integrity", "cyber-attack",
    "nuclear OR chemical OR biological AND weapon", "weapons of mass destruction", "proliferation of arms",
    "disarmament", "terrorist* OR terrorist group OR terrorism", "drug trafficking", "corruption", "human trafficking",
    "sanctions busting", "violation of sanction*"
]


In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON


# Define the SPARQL endpoint
sparql_endpoint = "http://publications.europa.eu/webapi/rdf/sparql"

# Initialize the SPARQL wrapper
sparql = SPARQLWrapper(sparql_endpoint)


In [26]:
# Define the SPARQL query template
sparql_query= PREFIX eurlex: <http://publications.europa.eu/ontology/euvoc#>
SELECT ?document ?celexNumber
WHERE {
  ?document eurlex:hasCelexNumber ?celexNumber .
  ?document eurlex:title ?title .
  FILTER regex(?title, "sanction", "i") || regex(?title, "war", "i")
}


SyntaxError: invalid syntax (3005008529.py, line 2)

In [25]:
results["results"]["bindings"]

[]

In [24]:
celex_ids = []
sparql_query = sparql_query_template.format('sanction')
print(sparql_query)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results)
#celex_id = results["celexNumber"]["value"]
#print(celex_id)
for result in results["head"]["vars"]:
        celex_id = result["celexNumber"]["value"]
        celex_ids.append(celex_id)
celex_id


    PREFIX eurlex: <http://publications.europa.eu/ontology/euvoc#>
    SELECT DISTINCT ?celexNumber
    WHERE {
        ?document eurlex:hasCelexNumber ?celexNumber .
        ?document eurlex:title ?title .
        FILTER regex(?title, "sanction", "i")
    }

{'head': {'link': [], 'vars': ['celexNumber']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}


TypeError: string indices must be integers, not 'str'

In [7]:
# Execute the queries for each keyword
celex_ids = []
for keyword in keywords:
    sparql_query = sparql_query_template.format(keyword)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        celex_id = results["celexNumber"]["value"]
        celex_ids.append(celex_id)

In [9]:
celex_ids

[]

In [ ]:
# Deduplicate CELEX IDs
celex_ids = list(set(celex_ids))

# Print the CELEX IDs
print("CELEX IDs:")
for celex_id in celex_ids:
    print(celex_id)


In [ ]:
#take the CELEX IDs queried through the keyword searches and download all documents
import eurlex
from eurlex import get_html_by_celex_id, parse_html

# Retrieve and parse the document with CELEX ID "32019R0947" into a Pandas DataFrame
celex_id = data["CELEX_id"].unique()

for i in celex_id:
    html = get_html_by_celex_id(celex_id)
    df = parse_html(html)
    corpus.append(df)

In [12]:
# Get the first line of Article 1
df_article_1 = df[df.article == "1"]
df_article_1_line_1 = df_article_1.iloc[0]

# Display the subtitle of Article 1
print(df_article_1_line_1.article_subtitle)


# Display the corresponding text
print(df_article_1_line_1.text)

ImportError: cannot import name 'get_html_by_celex_id' from 'eurlex' (C:\Users\cecilianatalie.strom\bin\conda\Lib\site-packages\eurlex\__init__.py)

**Option 2 Python EU lex SPARQL query (https://pypi.org/project/pyeurlex/)**

In [13]:
from eurlex import Eurlex
eur = Eurlex()

ImportError: cannot import name 'Eurlex' from 'eurlex' (C:\Users\cecilianatalie.strom\bin\conda\Lib\site-packages\eurlex\__init__.py)

In [ ]:
#create a query based on the advanced expert search SPARQL query format of the eulex website
q = eur.make_query(resource_type = "caselaw", order = True, limit = 10)
print(q)

In [ ]:
d = eur.query_eurlex(q)  # where q is a query generated in a previous step or a string defined by you
print(d)

In [ ]:
PREFIX eurlex: <http://publications.europa.eu/ontology/euvoc#>

SELECT ?document ?celexNumber
WHERE {
  ?document eurlex:hasCelexNumber ?celexNumber .
  ?document eurlex:title ?title .
  FILTER regex(?title, "sanction", "i") || regex(?title, "war", "i")
}


*This will return a pandas data frame of the results. Its columns depend on the the fields that you included. 
Once you pick a single url or identifier from the df, you can download a notice or data based on that indentifier. To download the notices as xml, use download_xml() as below.*


*This step will need to be done in a loop to access and download all the files*

In [ ]:
x = eur.download_xml("32014R0001", notice="tree") # without the file parameter to specify the filename, the celex number will be used.
print(x)

#get thedata
d = eur.get_data("http://publications.europa.eu/resource/celex/32016R0679", type="text")
print(d)

**Option3: SPARQL query via EU Lex Expert Search**

**Option 4: LEGALBERT**